# **<mark>_ECOMERCE GOLD LAYER DATA PIPELINE_ </mark>**

#### **Load Necessary Modules**

In [18]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


StatementMeta(, 23e4137f-c061-42ab-a875-b3f4355165cf, 21, Finished, Available, Finished, False)

#### **Merge ALL The Silver Layer Ecomerce Tables**

In [37]:
# Load all the table from silver layer
cust = spark.sql("select * from Silver.dbo.tb_customers").alias("c")
orders = spark.sql("select * from Silver.dbo.tb_orders").alias("o")
payments = spark.sql("select * from Silver.dbo.tb_payments").alias("p")
support = spark.sql("select * from Silver.dbo.tb_support_ticket").alias("s")
web = spark.sql("select * from Silver.dbo.tb_web_activities").alias("w")

df_customer360 =(
    cust
        .join(orders, col("c.customer_id") == col("o.customer_id"), "left")
        .join(payments, col("c.customer_id") == col("p.customer_id"), "left")
        .join(support, col("c.customer_id") == col("s.customer_id"), "left")
        .join(web, col("c.customer_id") == col("w.customer_id"), "left")
        .select(
            col("c.customer_id"),
            col("c.name"),
            col("c.email"),
            col("c.gender"),
            col("c.dob"),
            col("c.location"),

            col("o.order_id"),
            col("o.order_date"),
            col("o.amount").alias("order_amount"),
            col("o.status"),
            
            col("p.payment_method"),
            col("p.payment_status"),
            col("p.amount").alias("payment_amount"),

            col("s.ticket_id"),
            col("s.issue_type"),
            col("s.ticket_date"),
            col("s.resolution_status"),

            col("w.page_viewed"),
            col("w.device_type"),
            col("w.session_time")
        )
)

StatementMeta(, 23e4137f-c061-42ab-a875-b3f4355165cf, 40, Finished, Available, Finished, False)

#### **Store Final Merge Table Into Gold Layer**

In [38]:
df_customer360.write.format("delta").mode("overwrite").saveAsTable("Gold.dbo.tb_customer_360")

StatementMeta(, 23e4137f-c061-42ab-a875-b3f4355165cf, 41, Finished, Available, Finished, True)